In [1]:
# Use autoplot 207's code for this app
import datetime
import sys

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 9
TITLE = "21-22 January 2025"
SUBTITLE = "8 AM 23 January 2025"
SETPOINT_LOCS = {}
sts = utc(2025, 1, 22, 20)
ets = utc(2025, 1, 23, 18)
# Get available data
ctx = {
    "coop": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            "Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
            "IEM 2022-2023 Winter Storm #%s"
        )
        % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,MI,APX,14.0,-85.840000,44.730000,POINT (1111487.946 67477.686),True,0,True,LSR,28,20
1,MI,GRR,14.0,-86.180000,43.370000,POINT (1110540.849 -86752.399),True,1,True,LSR,28,15
2,MI,GRR,12.0,-86.160000,43.130000,POINT (1116572.13 -112884.625),True,2,True,LSR,28,14
3,MI,APX,11.0,-85.260000,45.070000,POINT (1149546.938 112954.16),True,3,True,LSR,29,21
4,MI,APX,10.2,-85.990000,44.720000,POINT (1100053.023 64325.961),True,4,True,LSR,27,20
...,...,...,...,...,...,...,...,...,...,...,...,...
407,Z,NaN,0.0,-96.739139,44.117971,POINT (260227.691 -92858.648),True,Z20,False,NaN,0,15
408,Z,NaN,0.0,-95.613014,44.079929,POINT (350227.691 -92858.648),True,Z21,False,NaN,3,15
409,Z,NaN,0.0,-94.488339,44.030706,POINT (440227.691 -92858.648),True,Z22,False,NaN,6,15
410,Z,NaN,0.0,-93.365479,43.970330,POINT (530227.691 -92858.648),True,Z23,False,NaN,9,15


In [8]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'Z19',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250123.png")
    mp.close()


main()

    state  wfo     val        lon        lat                             geo  \
34     IA  DVN  4.0000 -91.690000  42.040000  POINT (685026.168 -294366.453)   
43     IA  DVN  3.5000 -91.640000  42.020000  POINT (689350.435 -296162.409)   
45     IA  DVN  3.4000 -91.580000  42.040000  POINT (694054.501 -293441.184)   
46     IA  DVN  3.3000 -91.710000  41.980000  POINT (684039.643 -301178.851)   
47     IA  DVN  3.3000 -91.110000  42.080000   POINT (732143.686 -284922.99)   
49     IA  DVN  3.2000 -91.660000  42.010000  POINT (687818.213 -297437.909)   
52     IA  DVN  3.2000 -91.630000  42.030000  POINT (690061.188 -294970.771)   
54     IA  DVN  3.0000 -90.670000  42.070000  POINT (768313.071 -282001.259)   
55     IA  DVN  3.0000 -91.420000  41.490000  POINT (713378.148 -352973.038)   
56     IA  DVN  3.0000 -90.920000  41.860000  POINT (750341.807 -307556.418)   
57     IA  DVN  3.0000 -91.880000  42.090000    POINT (668892.98 -290397.29)   
59     IA  DVN  3.0000 -91.650000  41.99